# Riot API for team fight tactics

## Get puuid

In [ ]:
import pandas as pd

In [54]:
import requests
api_key = 'api_key'

name = 'A chess player'

url = f"https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/{name}?api_key={api_key}"

r = requests.get(url)

json=r.json()
puuid=json["puuid"]

## Get match_ids which the puuid gotten has

In [110]:
url = f"https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count=100&api_key={api_key}"
r1 = requests.get(url)

match_ids = r1.json()

In [113]:
len(match_ids)

58

In [114]:
match_ids

['KR_5907352652',
 'KR_5907304689',
 'KR_5907216675',
 'KR_5907151064',
 'KR_5907112165',
 'KR_5906968447',
 'KR_5906963868',
 'KR_5906859491',
 'KR_5906834768',
 'KR_5905106071',
 'KR_5905031639',
 'KR_5904943972',
 'KR_5904910525',
 'KR_5904757498',
 'KR_5904762488',
 'KR_5904658565',
 'KR_5904652629',
 'KR_5904635633',
 'KR_5904552508',
 'KR_5904456956',
 'KR_5904531139',
 'KR_5904358858',
 'KR_5904394753',
 'KR_5904289921',
 'KR_5904314699',
 'KR_5894619646',
 'KR_5894575111',
 'KR_5894530058',
 'KR_5894454015',
 'KR_5894379593',
 'KR_5894200104',
 'KR_5893846153',
 'KR_5893970940',
 'KR_5892255809',
 'KR_5892310851',
 'KR_5892195421',
 'KR_5892129913',
 'KR_5892104503',
 'KR_5890168990',
 'KR_5890163847',
 'KR_5889898439',
 'KR_5889963678',
 'KR_5889829586',
 'KR_5889855047',
 'KR_5889851424',
 'KR_5887329243',
 'KR_5887342350',
 'KR_5887247018',
 'KR_5886794223',
 'KR_5886821364',
 'KR_5886589477',
 'KR_5886477915',
 'KR_5886625860',
 'KR_5886524624',
 'KR_5886513365',
 'KR_58865

## Overlap_checking

In [339]:
def overlap_check(match_id,api_key,puuid):

    '''
    Arguments 
    match_id : The match id 
    api_key : The key used to access riot api.
    puuid : A user's puuid
    '''
    print(puuid)
    
    url = f"https://asia.api.riotgames.com/tft/match/v1/matches/{match_id}?api_key={api_key}"
    r2 = requests.get(url)

    example = r2.json()
    info=example['info']

    synergy = pd.DataFrame()
    status = pd.DataFrame()
    augments = pd.DataFrame()

    for case in info['participants'] :
        augment = pd.DataFrame({'augments' : case['augments'],
        })
        augment['puuid'] = case['puuid']

        augments = pd.concat([augments,augment])

        
        d={'puuid' : case['puuid'] , 
        'gold_left' : case['gold_left'],
        'last_round' :case['last_round'],
        'placement' : case['placement'],
        'players_eliminated': case['players_eliminated'],
        'total_damage_to_players': case['total_damage_to_players']}
        
        stat =pd.DataFrame(d,index=[0])
        status = pd.concat([status , stat])
        A=pd.DataFrame(case['traits'])
        A['puuid'] = case['puuid']
        synergy = pd.concat([synergy,A]) 
        

    status = status.reset_index(drop = True)
    synergy = synergy.reset_index(drop = True)
    augments = augments.reset_index(drop = True)    


    my_synergy_activated = synergy.loc[(synergy.puuid == puuid).astype('int') + (synergy.tier_current >= 1).astype('int') == 2,:]
    #print(my_synergy_activated)
    selected=my_synergy_activated.loc[my_synergy_activated.num_units >= my_synergy_activated.num_units.max(),:]
    
    # The number of synergies which had the largest number of units can be more than 2.
    
    if len(selected) == 0 :
        overlapped = pd.DataFrame({'name' : 'empty' ,'count' : 0 , 'placement' : status.loc[status.puuid == puuid , 'placement'].values[0]},index=[0])
    else :        
        overlapped = synergy.groupby(['name'])['num_units'].agg('count').reset_index()
        overlapped.columns = ['name', 'count']
        overlapped = overlapped.loc[overlapped.name == selected['name'].iloc[0],:]
        
        overlapped['placement'] = status.loc[status.puuid == puuid , 'placement'].values[0]
    return overlapped

In [336]:
overlap_check(id,api_key,puuid)

RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA


,name,num_units,placement
0,empty,0,3


In [431]:
overlap_check_df = pd.DataFrame()
for id in match_ids :
    overlap_check_df = pd.concat([overlap_check_df , overlap_check(id,api_key,puuid)])
    

RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpE

In [432]:
overlap_check_df.loc[overlap_check_df.placement <=4,'under4'] = 'Y'
overlap_check_df.loc[overlap_check_df.placement > 4,'under4'] = 'N'

In [435]:
overlap_check_df

,name,count,placement,under4
15,Set6_Mutant,4,6,N
15,Set6_Mutant,4,5,N
3,Set6_Bruiser,4,4,Y
2,Set6_Bodyguard,5,3,Y
0,empty,0,3,Y
15,Set6_Mutant,6,5,N
3,Set6_Bruiser,3,5,N
23,Set6_Yordle,5,5,N
3,Set6_Bruiser,4,4,Y
24,Set6_Yordle,5,4,Y


In [436]:
'''
number_players_overlapping
- The number of players who overlap a player's main synergy in each match.
'''

overlap_check_df.columns = ['name','num_players_overlapping','placement','under4']

In [440]:
overlap_check_df

,name,num_players_overlapping,placement,under4
15,Set6_Mutant,4,6,N
15,Set6_Mutant,4,5,N
3,Set6_Bruiser,4,4,Y
2,Set6_Bodyguard,5,3,Y
0,empty,0,3,Y
15,Set6_Mutant,6,5,N
3,Set6_Bruiser,3,5,N
23,Set6_Yordle,5,5,N
3,Set6_Bruiser,4,4,Y
24,Set6_Yordle,5,4,Y


In [441]:
overlap_check_df.groupby(['name','under4']).agg({'num_players_overlapping' : 'mean', 'placement' : ['mean','count']})

num_players_overlapping placement      
                                          mean      mean count
name            under4                                        
Set6_Arcanist   N                     5.000000  6.000000     1
                Y                     4.000000  2.500000     2
Set6_Bodyguard  N                     5.000000  7.000000     1
                Y                     5.000000  3.000000     1
Set6_Bruiser    N                     4.333333  5.833333     6
                Y                     3.875000  3.375000     8
Set6_Challenger N                     3.750000  7.500000     4
                Y                     3.000000  2.000000     1
Set6_Debonair   Y                     3.333333  3.000000     3
Set6_Hextech    Y                     4.000000  3.000000     2
Set6_Innovator  N                     3.333333  6.333333     3
                Y                     3.500000  3.000000     2
Set6_Mutant     N                     4.750000  5.500000     4
                Y                     3.000000  2.000000     1
Set6_Scholar    N                     4.000000  5.000000     1
Set6_Socialite  Y                     5.000000  2.000000     1
Set6_Striker    N                     4.000000  7.000000     1
Set6_Syndicate  N                     3.000000  5.000000     1
                Y                     4.000000  3.000000     2
Set6_Twinshot   Y                     4.000000  4.000000     1
Set6_Yordle     N                     5.000000  5.666667     3
                Y                     3.375000  2.750000     8
empty           Y                     0.000000  3.000000     1

### Result
- From the above, 
    - Yodle can give me higer score than others. However, the synergy can't be a good solution when more than 4 players choose the synergy.
    - I chose Yodle synergy frequently.
    - I were good at the deck based on Mutants.
    - Brusier synergy doesn't always guarantee high rank in a match.

## Augments

In [391]:
def augment_and_placement(match_id,api_key,puuid):

    '''
    Arguments 
    match_id : The match id 
    api_key : The key used to access riot api.
    puuid : A user's puuid
    '''
    print(puuid)
    
    url = f"https://asia.api.riotgames.com/tft/match/v1/matches/{match_id}?api_key={api_key}"
    r2 = requests.get(url)

    example = r2.json()
    info=example['info']

    synergy = pd.DataFrame()
    status = pd.DataFrame()
    augments = pd.DataFrame()

    for case in info['participants'] :
        augment = pd.DataFrame({'augments' : case['augments'],
        })
        augment['puuid'] = case['puuid']
        augment['placement'] = case['placement']

        augments = pd.concat([augments,augment])

        
        d={'puuid' : case['puuid'] , 
        'gold_left' : case['gold_left'],
        'last_round' :case['last_round'],
        'placement' : case['placement'],
        'players_eliminated': case['players_eliminated'],
        'total_damage_to_players': case['total_damage_to_players']}
        
        stat =pd.DataFrame(d,index=[0])
        status = pd.concat([status , stat])
        A=pd.DataFrame(case['traits'])
        A['puuid'] = case['puuid']
        synergy = pd.concat([synergy,A]) 
        

    status = status.reset_index(drop = True)
    synergy = synergy.reset_index(drop = True)
    augments = augments.reset_index(drop = True)    
    
    a_match_augments =augments.loc[augments.puuid == puuid,:]
    a_match_augments.loc[:,'match_id'] = match_id
    return a_match_augments


In [395]:
augment_and_placement_df = pd.DataFrame()
for id in match_ids :
    augment_and_placement_df = pd.concat([augment_and_placement_df,augment_and_placement(id,api_key,puuid)])
    

RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA


/home/saesimcheon/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/saesimcheon/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpEGpSVaKpb4VKvdobiSYfA1CiSkA
RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3G8cbpE

In [400]:
pd.set_option('display.max_rows', 500)

In [414]:
augment_and_placement_df.loc[augment_and_placement_df.placement <= 4, 'under4'] = 'Y'
augment_and_placement_df.loc[augment_and_placement_df.placement > 4 , 'under4'] = 'N'

In [415]:
augment_and_placement_df

,augments,puuid,placement,match_id,under4
0,TFT6_Augment_ExperimentalTrait,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,6,KR_5907352652,N
1,TFT6_Augment_Phalanx1,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,6,KR_5907352652,N
2,TFT6_Augment_CyberneticUplink2,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,6,KR_5907352652,N
18,TFT6_Augment_GachaAddict,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,5,KR_5907304689,N
19,TFT6_Augment_MakeshiftArmor1,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,5,KR_5907304689,N
20,TFT6_Augment_ColossusArmorPlating,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,5,KR_5907304689,N
0,TFT6_Augment_EnforcerSwiftJustice,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,4,KR_5907216675,Y
1,TFT6_Augment_TinyTitans,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,4,KR_5907216675,Y
2,TFT6_Augment_SecondWind2,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,4,KR_5907216675,Y
15,TFT6_Augment_HeroTrait,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,3,KR_5907151064,Y


In [427]:
augment_and_placement_df

,augments,puuid,placement,match_id,under4
0,TFT6_Augment_ExperimentalTrait,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,6,KR_5907352652,N
1,TFT6_Augment_Phalanx1,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,6,KR_5907352652,N
2,TFT6_Augment_CyberneticUplink2,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,6,KR_5907352652,N
18,TFT6_Augment_GachaAddict,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,5,KR_5907304689,N
19,TFT6_Augment_MakeshiftArmor1,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,5,KR_5907304689,N
20,TFT6_Augment_ColossusArmorPlating,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,5,KR_5907304689,N
0,TFT6_Augment_EnforcerSwiftJustice,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,4,KR_5907216675,Y
1,TFT6_Augment_TinyTitans,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,4,KR_5907216675,Y
2,TFT6_Augment_SecondWind2,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,4,KR_5907216675,Y
15,TFT6_Augment_HeroTrait,RGEFgkuOuFYvLAWKTSM1XL_0uD--bKVmcPfQBS706288F3...,3,KR_5907151064,Y


In [428]:
groupped = augment_and_placement_df.groupby(['augments','under4']).agg({'placement' : 'mean','puuid':'count'})

In [429]:
groupped = groupped.reset_index()

In [430]:
groupped

,augments,under4,placement,puuid
0,TFT6_Augment_ArcanistRunicShield1,Y,4.000000,1
1,TFT6_Augment_ArcanistRunicShield2,N,5.000000,1
2,TFT6_Augment_ArcanistRunicShield2,Y,4.000000,1
3,TFT6_Augment_ArcanistSpellBlade,Y,2.000000,1
4,TFT6_Augment_Archangel2,N,6.000000,3
5,TFT6_Augment_Archangel2,Y,3.000000,4
6,TFT6_Augment_Ascension,Y,1.500000,2
7,TFT6_Augment_Backfoot1,Y,2.000000,1
8,TFT6_Augment_Backfoot2,N,6.000000,1
9,TFT6_Augment_Backfoot2,Y,4.000000,1


In [412]:
groupped.loc[groupped.placement > 4 , :]

,augments,placement,puuid,match_id
2,TFT6_Augment_ArcanistRunicShield2,5,1,1
7,TFT6_Augment_Archangel2,5,1,1
8,TFT6_Augment_Archangel2,6,1,1
9,TFT6_Augment_Archangel2,7,1,1
14,TFT6_Augment_Backfoot2,6,1,1
17,TFT6_Augment_BinaryAirdrop,7,1,1
18,TFT6_Augment_BrawlerHPRegen1,5,1,1
19,TFT6_Augment_BruteEmblem2,6,1,1
26,TFT6_Augment_CelestialBlessing3,6,2,2
27,TFT6_Augment_ChallengerEmblem,8,1,1


In [408]:
groupped.index.

MultiIndex([('TFT6_Augment_ArcanistRunicShield1', 4),
            ('TFT6_Augment_ArcanistRunicShield2', 4),
            ('TFT6_Augment_ArcanistRunicShield2', 5),
            (  'TFT6_Augment_ArcanistSpellBlade', 2),
            (          'TFT6_Augment_Archangel2', 2),
            (          'TFT6_Augment_Archangel2', 3),
            (          'TFT6_Augment_Archangel2', 4),
            (          'TFT6_Augment_Archangel2', 5),
            (          'TFT6_Augment_Archangel2', 6),
            (          'TFT6_Augment_Archangel2', 7),
            ...
            (          'TFT6_Augment_Traitless2', 3),
            (      'TFT6_Augment_TreasureTrove1', 5),
            (      'TFT6_Augment_TreasureTrove1', 7),
            (           'TFT6_Augment_TriForce1', 3),
            (           'TFT6_Augment_TriForce2', 2),
            (           'TFT6_Augment_TriForce2', 5),
            (              'TFT6_Augment_Twins2', 7),
            (            'TFT6_Augment_Weakspot', 2),
            

### The number of samples is not enough to analyze augments.

# Future work

- Get and analyze  high tier users' match information.
- Analyze what the synergy and augments pair affects for a player's rank in a game.
- Augments pair.